In [2]:
#from google.colab import drive
#drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#%cd /content/drive/MyDrive/Masters_Project/Datasets/OASIS3/

/content/drive/.shortcut-targets-by-id/1bqqm9-0FMOqiqyH_NIQBh-lJ36pWa7aX/Masters_Project/Datasets/OASIS3


In [1]:
!ls

1-oasis3-t1w-cdr-split.ipynb	3-Save_ad-jpg-slices.ipynb  Test-Ganomaly.ipynb
2-Save_non-ad-jpg-slices.ipynb	4-load_oasis3_data.ipynb    ganomaly


In [2]:
!pip install -r ./ganomaly/requirements.txt

     |████████████████████████████████| 157 kB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 430 kB 18.7 MB/s eta 0:00:01
     |████████████████████████████████| 2.3 MB 6.1 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 278 kB 9.7 MB/s eta 0:00:01
     |████████████████████████████████| 90 kB 9.4 MB/s  eta 0:00:01
     |████████████████████████████████| 13.1 MB 12.1 MB/s eta 0:00:01
     |████████████████████████████████| 17.3 MB 9.2 MB/s eta 0:00:01
     |████████████████████████████████| 112 kB 9.4 MB/s eta 0:00:01
     |████████████████████████████████| 158 kB 10.9 MB/s eta 0:00:01
     |████████████████████████████████| 53 kB 3.4 MB/s  eta 0:00:01
     |████████████████████████████████| 62 kB 1.8 MB/s  eta 0:00:01
     |████████████████████████████████| 226 kB 7.3 MB/s eta 0:00:01
     |████████████████████████████████| 510 kB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 

  Attempting uninstall: kiwisolver
    Found existing installation: kiwisolver 1.3.1
    Uninstalling kiwisolver-1.3.1:
      Successfully uninstalled kiwisolver-1.3.1
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 2.4.7
    Uninstalling pyparsing-2.4.7:
      Successfully uninstalled pyparsing-2.4.7
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.4
    Uninstalling numpy-1.19.4:
      Successfully uninstalled numpy-1.19.4
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.1
    Uninstalling python-dateutil-2.8.1:
      Successfully uninstalled python-dateutil-2.8.1
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.3.3
    Uninstalling matplotlib-3.3.3:
      Successfully uninstalled matplotlib-3.3.3
  Attempting uninstall: pyzmq
    Found existing installation: pyzmq 20.0.0
    Uninstalling pyzmq-20.0.0:
      Successfully uninstalled pyzmq-20.0.0
  Attempti

In [3]:
!pip install mkl-fft

     |████████████████████████████████| 232 kB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 6.1 MB 4.7 MB/s eta 0:00:01
     |████████████████████████████████| 425 kB 5.7 MB/s eta 0:00:01
     |████████████████████████████████| 396 kB 8.4 MB/s eta 0:00:01
     |████████████████████████████████| 737.3 MB 5.3 kB/s  eta 0:00:01    |████▌                           | 102.4 MB 6.7 MB/s eta 0:01:35     |██████████████████████████▍     | 608.3 MB 13.6 MB/s eta 0:00:10     |███████████████████████████▊    | 638.7 MB 7.6 MB/s eta 0:00:14
     |████████████████████████████████| 11.2 MB 12.1 MB/s eta 0:00:01
     |████████████████████████████████| 3.6 MB 5.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.9 MB 5.8 MB/s eta 0:00:01
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages wit

In [4]:
!pip install Pillow

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


# imports

In [9]:
import os
import glob
import torch
import numpy as np
import torchvision.datasets as datasets
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms


In [10]:
from PIL import Image
#import cv2
import os.path
import numpy as np
import pickle
from typing import Any, Callable, Optional, Tuple

from torchvision.datasets import VisionDataset
from torchvision.datasets.utils import check_integrity, download_and_extract_archive

In [11]:
print(torch.__version__)

1.2.0


In [12]:
import torchvision
torchvision.__version__

'0.4.0'

# Create Oasis3 data class

In [13]:
transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

In [19]:
train_image_number = 140 # Image slice from MRI scans to use for training data

base_folder = '/pyt_oasis3'
non_ad_jpg_folder = '/tmp/data/t1w_non_ad_jpgs/' + str(train_image_number) + '/'
ad_jpg_folder = '/tmp/data/t1w_ad_jpgs/'

class Oasis3_train_normal(VisionDataset):
    def __init__(self,
            root: str,
            train: bool = True, # Train or test dataset
            transform: Optional[Callable] = None,
            target_transform: Optional[Callable] = None) -> None:
        super(Oasis3_train_normal, self).__init__(root, transform=transform, target_transform=target_transform)
        self.train = train # Train or test set
        
        self.data = []
        self.targets = []
        
        for file_path in glob.glob(non_ad_jpg_folder+'**.jpg'):
            with open (file_path, 'rb') as f:
                # image needs to be a PIL image
                img = Image.open(f)
                # Resize all images 176, 256, 3 -> 256, 256, 0
                dsize = (256, 256)
                resized = img.resize(dsize)
                self.data.append(resized)
                self.targets.extend(str(0)) # based on the jpg_folder in for loop
        self.data = np.vstack(self.data).reshape(-1, 256, 256)
        #self.data = self.data.transpose((0, 2, 3, 1))  # convert to HWC
        self.data = self.data.transpose((0, 1, 2))  # convert to HWC

    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        img, target = self.data[index], self.targets[index]
        
        img = Image.fromarray(img)
        
        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return (img, target)

In [20]:
# Issue is i need to make the data transformable inside __getitem__
a=Oasis3_train_normal(base_folder, False, transform=transform)

In [16]:
# OpenCV ---
# Original: (176, 256, 3)
# Re-shaped up: (3, 256, 256)
# After transform: (3, 256, 256)
# PIL ---
# Shape : (256, 256)
a.data[0].shape

(256, 256)

In [21]:
# The class should be index into which invokes the __getitem__ call
a[0].shape

RuntimeError: output with shape [1, 256, 256] doesn't match the broadcast shape [3, 256, 256]

# PIL image reading

In [96]:
data = []
with open ('./t1w_non_ad_jpgs/140/OAS30074_ses-d0049_run-01_T1w.jpg', 'rb') as f:
  img = Image.open(f)
  dsize = (256, 256)
  resized = img.resize(dsize)
  data.append(resized)

In [102]:
data[0].size

(256, 256)

# CIFAR data representation testing

In [55]:
from torchvision.datasets import CIFAR10

In [57]:
transform = transforms.Compose(
    [
        transforms.Resize(32),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

dataset = {}
dataset['train'] = CIFAR10(root='./', train=True, download=True, transform=transform)



0it [00:00, ?it/s]

  0%|          | 0/170498071 [00:00<?, ?it/s]



  0%|          | 57344/170498071 [00:00<05:29, 516692.23it/s]

  0%|          | 286720/170498071 [00:00<04:14, 668180.87it/s]

  1%|          | 1171456/170498071 [00:00<03:03, 922090.63it/s]

  2%|▏         | 4022272/170498071 [00:00<02:08, 1299237.00it/s]

  5%|▌         | 9019392/170498071 [00:00<01:28, 1834651.66it/s]

  8%|▊         | 14475264/170498071 [00:00<01:00, 2580404.68it/s]

 12%|█▏        | 20045824/170498071 [00:00<00:41, 3608584.65it/s]

 15%|█▌        | 25583616/170498071 [00:01<00:28, 5003845.87it/s]

 18%|█▊        | 31039488/170498071 [00:01<00:20, 6855204.30it/s]

 21%|██▏       | 36577280/170498071 [00:01<00:14, 9258802.64it/s]

 25%|██▍       | 42213376/170498071 [00:01<00:10, 12285662.64it/s]

 28%|██▊       | 47898624/170498071 [00:01<00:07, 15961697.38it/s]

 32%|███▏      | 53747712/170498071 [00:01<00:05, 20414458.50it/s]

 35%|███▍      | 58990592/170498071 [00:01<00:04, 24991985.35it/s]

 38%|███▊      | 64495616/170498071 [00:01<00:03, 29816226.17it/s]


Extracting ./cifar-10-python.tar.gz to ./


In [59]:
type(dataset['train'])

torchvision.datasets.cifar.CIFAR10

In [65]:
elt = dataset['train'][0]

In [71]:
type(elt)

tuple

In [76]:
elt[0].shape

torch.Size([3, 32, 32])